In [8]:
from grafo import Grafo
import pandas as pd

In [9]:
cruces = pd.read_csv("data/cruces_clean.csv")
calles = pd.read_csv("data/direcciones_clean.csv")
g = Grafo(True)

In [10]:
cruces.drop_duplicates(["x", "y"])

,id_via,nombre_via,id_via_cruzada,nombre_via_cruzada,x,y
0,127,CALLE DE ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA,44260746,448479753
1,127,CALLE DE ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU,44260732,448479748
2,127,CALLE DE ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU,44290433,448494484
3,127,CALLE DE ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE,44320997,448510385
4,127,CALLE DE ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN,44322285,448530840
...,...,...,...,...,...,...
36151,31003581,GLORIETA DE JOSÉ ANTONIO JIMÉNEZ SALAS,31001595,AVENIDA DE JUAN ANTONIO SAMARANCH,44819509,448232109
36154,31003584,GLORIETA DE JOSÉ MARÍA GARCÍA DE PAREDES,31001596,AVENIDA DE JOSÉ ANTONIO CORRALES,44793495,448200899
36159,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ,31003706,CALLE DE TREN OBRERO,44083855,446665589
36160,31003706,CALLE DE TREN OBRERO,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ,44083665,446665463


In [11]:
calles["orden"] = calles["Direccion completa"].str.replace(r"\d+", lambda m: chr(int(m.group(0))), regex=True)
calles

,Direccion completa,Codigo de via,x,y,orden
0,AUTOV A-1 1000 EN,31001337,44305633,448250340,AUTOV A- Ϩ EN
1,AUTOV A-1 1000 SA,31001337,44312246,448249077,AUTOV A- Ϩ SA
2,AUTOV A-1 1100 EN,31001337,44367522,448330933,AUTOV A- ь EN
3,AUTOV A-1 1100 SA,31001337,44373755,448325151,AUTOV A- ь SA
4,AUTOV A-1 1200 EN,31001337,44425207,448408185,AUTOV A- Ұ EN
...,...,...,...,...,...
210352,PASEO ZURRÓN 49,725,45051964,448025888,PASEO ZURRÓN 1
210353,PASEO ZURRÓN 51,725,45058080,448025960,PASEO ZURRÓN 3
210354,PASEO ZURRÓN 53,725,45061025,448027575,PASEO ZURRÓN 5
210355,PASEO ZURRÓN 55,725,45063500,448027080,PASEO ZURRÓN 7


In [12]:
calles.drop_duplicates("Codigo de via")

,Direccion completa,Codigo de via,x,y,orden
0,AUTOV A-1 1000 EN,31001337,44305633,448250340,AUTOV A- Ϩ EN
8,AUTOV A-2 500 EN,31001336,44469993,447782563,AUTOV A- Ǵ EN
37,AUTOV A-3 400 EN,31001342,44439783,447321577,AUTOV A- Ɛ EN
77,AUTOV A-4 400 EN,31001334,44167422,447063954,AUTOV A- Ɛ EN
96,AUTOV A-42 500 EN,31001341,43905717,447000585,AUTOV A-* Ǵ EN
...,...,...,...,...,...
210105,ZURBANO 1,807000,44124688,447560119,ZURBANO 
210206,ZURBARÁN 1,807100,44106724,447590046,ZURBARÁN 
210233,ZURICH 1,2462,44840460,447486805,ZURICH 
210258,ZURITA 1,807300,44088069,447375916,ZURITA 


In [13]:
def asociar(row: pd.Series):
    df = calles[calles["Codigo de via"] == row["id_via"]]
    if df.empty:
        return "None"
    df["distance"] = df.apply(lambda row2: (row["x"]-row2["x"])**2+(row["y"]-row2["y"])**2, axis=1)
    return df.loc[int(df["distance"].idxmax())]["orden"]

In [14]:
cruces["direccion_asociada"] = cruces.apply(asociar, axis=1)
cruces

,id_via,nombre_via,id_via_cruzada,nombre_via_cruzada,x,y,direccion_asociada
0,127,CALLE DE ISABEL COLBRAND,2792,CALLE DE CASTIELLO DE JACA,44260746,448479753,ISABEL COLBRAND 
1,127,CALLE DE ISABEL COLBRAND,1837,CALLE DE FEDERICO MOMPOU,44260732,448479748,ISABEL COLBRAND 
2,127,CALLE DE ISABEL COLBRAND,87100,CALLE DE MARÍA TUBAU,44290433,448494484,ISABEL COLBRAND 
3,127,CALLE DE ISABEL COLBRAND,2793,PASEO DE TIERRA DE MELIDE,44320997,448510385,ISABEL COLBRAND  D
4,127,CALLE DE ISABEL COLBRAND,2794,CALLE DE PORTOMARÍN,44322285,448530840,ISABEL COLBRAND  D
...,...,...,...,...,...,...,...
36160,31003706,CALLE DE TREN OBRERO,31003703,CALLE DE JOSÉ MARÍA LOZANO SAINZ,44083665,446665463,TREN OBRERO 
36161,31004081,GLORIETA DE CIRILO MARTÍN MARTÍN,369600,CARRETERA DE HÚMERA,43390886,447883842,None
36162,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,29900,CALLE DE ALICANTE,44145890,447199432,ISIDORO ÁLVAREZ ÁLVAREZ  C
36163,31005685,CALLE DE ISIDORO ÁLVAREZ ÁLVAREZ,2729,CALLE DEL GRANITO,44150370,447189024,ISIDORO ÁLVAREZ ÁLVAREZ 


In [15]:
for _, cruce in cruces.drop_duplicates(["x", "y"]).iterrows():
    g.agregar_vertice((cruce["x"], cruce["y"]))

In [16]:
c = 0
for via in cruces["nombre_via"].unique():
    df = cruces[cruces["nombre_via"] == via]
    df = df.sort_values(by="direccion_asociada")
    for i in range(df.shape[0]-1):
        r1 = df.iloc[i]
        r2 = df.iloc[i+1]
        if (r1["x"], r1["y"]) != (r2["x"], r2["y"]):
            g.agregar_arista(s=(r1["x"], r1["y"]), t=(r2["x"], r2["y"]), data=via, weight=1) # Agregar peso
        c += 1

print(c)

28942


In [17]:
len(g.aristas)

23749

In [18]:
g.save_graph("plano_de_madrid.txt")